# Road Following 

If you've run through the collision avoidance sample, your should be familiar following three steps  
如果你已经完成过避障行为的notebook，那么你应该很熟悉以下三个步骤。

1.  Data collection  
    数据采集
2.  Training  
    训练
3.  Deployment  
    部署

In this notebook, we'll do the same exact thing!  Except, instead of classification, you'll learn a different fundamental technique, **regression**, that we'll use to
enable JetBot to follow a road (or really, any path or target point).  
在这个notebook中，我们会做同样的事情。除了分类之外，你还会学习另一种不同的基本方法，称之为**回归**，我们将使用它令JetBot能够跟随道路（或者说是跟随路径或者目标点）。

1. Place the JetBot in different positions on a path (offset from center, different angles, etc)
   把JetBot放置在路径上的不同位置（不同的角度不同的道路位置）

>  Remember from collision avoidance, data variation is key!
   请记住避免碰撞，多样的数据是关机

2. Display the live camera feed from the robot  
   显示机器人的实时摄像头图像
   
3. Using a gamepad controller, place a 'green dot', which corresponds to the target direction we want the robot to travel, on the image.  
   使用一个游戏手柄控制其，在图像上放置一个‘绿点’，它对应于我们希望机器人行进的目标方向。  
   
4. Store the X, Y values of this green dot along with the image from the robot's camera
   存储这个‘绿点’的x,y坐标以及机器人相机的图像

Then, in the training notebook, we'll train a neural network to predict the X, Y values of our label.  In the live demo, we'll use
the predicted X, Y values to compute an approximate steering value (it's not 'exactly' an angle, as
that would require image calibration, but it's roughly proportional to the angle so our controller will work fine).
然后，在训练的notebook中，我们会训练神经网络来预测我们标签的X，Y的值。在实时运行的notebook中，我们将通过计算X，Y的近似值进行转向控制。

So how do you decide exactly where to place the target for this example?  Here is a guide we think may help
那么你如何确定这个例子的确切位置呢？ 以下指南可能会帮到你


1.  Look at the live video feed from the camera
    查看摄像头的实时图像
2.  Imagine the path that the robot should follow (try to approximate the distance it needs to avoid running off road etc.)  
    想象一下机器人应该走的路径。（尽量接近它所需要的距离避免越出界限）
3.  Place the target as far along this path as it can go so that the robot could head straight to the target without 'running off' the road.  
    把目标尽可能远的路径上，以便机器人可以直接前往目标，而不会迷路。

> For example, if we're on a very straight road, we could place it at the horizon.  If we're on a sharp turn, it may need to be placed closer to the robot so it doesn't run out of boundaries.  
例如，如果我们在一条非常直的道路上，我们可以把它放在地平线上。 如果我们处于急转弯状态，可能需要将其放置在离JetBot较近的位置，这样它就不会超出边界。

Assuming our deep learning model works as intended, these labeling guidelines should ensure the following:
假设我们的深度学习模型按预期工作，以下这些标签指南应确保以下内容：


1.  The robot can safely travel directly towards the target (without going out of bounds etc.)  
    机器人可以安全地直接朝向目标（不出界等）

2.  The target will continuously progress along our imagined path  
    目标将沿着我们想象的道路不断前进

What we get, is a 'carrot on a stick' that moves along our desired trajectory.  Deep learning decides where to place the carrot, and JetBot just follows it :)
我们得到的是一根“胡萝卜棒”，沿着我们想要的轨迹移动。 深度学习决定胡萝卜放在哪里，JetBot就跟着它:)

### Labeling example video
### 如何标记视频演示

Execute the block of code to see an example of how to we labeled the images.  This model worked after only 123 images :)  
执行以下代码块以查看如何标记图像的示例。 这个模型仅用了123张图片:)

In [1]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FW4En6LejhI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


### Import Libraries
### 加载库

So lets get started by importing all the required libraries for "data collection" purpose. We will mainly use OpenCV to visualize and save image with labels. Libraries such as uuid, datetime are used for image naming.  
因此，让我们开始导入所有必需的库以进行“数据收集”。 我们将主要使用OpenCV来使用标签可视化和保存图像。 诸如uuid，datetime之类的库用于图像命名。

In [ ]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### Display Live Camera Feed
### 实时显示摄像头的图像

First, let's initialize and display our camera like we did in the teleoperation notebook. 
首先，让我们像在远程操作示例的notebook中一样初始化和显示我们的相机。

We use Camera Class from JetBot to enable CSI MIPI camera. Our neural network takes a 224x224 pixel image as input. We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task). In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.
我们使用JetBot的Camera Class来启用CSI MIPI相机。 我们的神经网络采用224x224像素图像作为输入。我们将相机设置为该大小，以最小化数据集的文件大小（我们已经测试它适用于此任务）。 在某些情况下，最好以较大的图像尺寸收集数据，然后缩小到所需的尺寸。

In [ ]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='steering')
throttle_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='throttle')

def display_steering(camera_image):
    image = np.copy(camera_image)
    steering = steering_slider.value
    throttle = throttle_slider.value
    x = int(steering * 224 / 2 + 112)
    y = int(throttle * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_steering)

display(widgets.HBox([image_widget, target_widget]), steering_slider, throttle_slider)

### Create Gamepad Controller
### 创建游戏手柄控制

This step is similar to "Teleoperation" task. In this task, we will use gamepad controller to label images.  
此步骤类似于“遥控操作notebook”任务。而在此任务中，我们将使用游戏手柄控制器来标记图像。

The first thing we want to do is create an instance of the Controller widget, which we'll use to label images with "x" and "y" values as mentioned in introduction. The Controller widget takes a index parameter, which specifies the number of the controller. This is useful in case you have multiple controllers attached, or some gamepads appear as multiple controllers. To determine the index of the controller you're using,  
我们要做的第一件事是创建一个Controller小部件的实例，我们将用它来标记带有“x”和“y”值的图像，如引言中所述。 Controller小部件采用索引参数，该参数指定控制器的编号。 如果您连接了多个控制器，或者某些游戏手柄显示为多个控制器，则此功能非常有用。 要确定您正在使用的控制器的索引。


Visit http://html5gamepad.com.  
访问http://html5gamepad.com  

Press buttons on the gamepad you're using
Remember the index of the gamepad that is responding to the button presses
Next, we'll create and display our controller using that index.  
按下您正在使用的游戏手柄上的按钮，记住响应按钮按下的游戏手柄的索引。接下来，我们将使用该索引创建和显示我们的控制器。

In [ ]:
controller = widgets.Controller(index=0)

display(controller)

### Connect Gamepad Controller to Label Images
### 将手柄实现图像标签功能

Now, even though we've connected our gamepad, we haven't yet attached the controller to label images! We'll connect that to the left and right vertical axes using the dlink function. The dlink function, unlike the link function, allows us to attach a transform between the source and target. 
现在，我们连接了游戏手柄，但还没有将控制器控制标签在图像上！ 我们将使用dlink功能将其连接到左右垂直轴。 与链接函数不同，dlink函数允许我们在源和目标之间附加变换。

In [ ]:
widgets.jsdlink((controller.axes[2], 'value'), (steering_slider, 'value'))
widgets.jsdlink((controller.axes[3], 'value'), (throttle_slider, 'value'))

### Collect data
### 收集数据

The following block of code will display the live image feed, as well as the number of images we've saved.  We store
the target X, Y values by
接下来的代码块会显示实时图像，以及我们保存图像的数量，存储的X，Y的值。

1. Place the green dot on the target  
   将绿点放在目标上
   
2. Press 'down' on the DPAD to save  
   按DPAD的的‘donw’保存

This will store a file in the ``dataset_xy`` folder with files named  

``xy_<x value>_<y value>_<uuid>.jpg``

这将会保存在``dataset_xy``文件夹中。并命名为``xy_<x value>_<y value>_<uuid>.jpg``

When we train, we load the images and parse the x, y values from the filename
当我们训练是，我们加载图像并从文件名中获得X，Y的值。

In [ ]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

for b in controller.buttons:
    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_steering(change):
    if change['new']:
        uuid = steering_throttle_uuid(steering_slider.value, throttle_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[13].observe(save_steering, names='value')

display(widgets.VBox([
    target_widget,
    count_widget
]))

### Next
### 下一步

Once you've collected enough data, we'll need to copy that data to our GPU desktop or cloud machine for training. First, we can call the following terminal command to compress our dataset folder into a single zip file.  

收集到足够的数据后，我们需要将这些数据复制到GPU平台或云计算机上进行训练。 首先，我们可以调用以下终端命令将我们的数据集文件夹压缩为单个zip文件。


> If you're training on the JetBot itself, you can skip this step!
> 如果你直接在JetBot上训练，你可以跳过此步骤。

The ! prefix indicates that we want to run the cell as a shell (or terminal) command.

The -r flag in the zip command below indicates recursive so that we include all nested files, the -q flag indicates quiet so that the zip command doesn't print any output

In [ ]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}

You should see a file named road_following_<Date&Time>.zip in the Jupyter Lab file browser. You should download the zip file using the Jupyter Lab file browser by right clicking and selecting Download.

解压完成，你会见到一个road_following_<Date&Time>.zip的文件